# Creating a Story Map Using Leaflet and CSVs

---

**Objectives:**

By the end of this exericse, you should be able to:

* Create an HTML page
* Host an interactive map (Leaflet) on GitHub Pages
* Create a storymap using Python, Leaflet, and CSV files

---

In this exercise, you will learn how to present spatial data in an interactive narrative similar to [ESRI StoryMaps](https://storymaps-classic.arcgis.com/en/gallery/#s=0) using open source products. You will learn how to host a live version of an HTML webpage on GitHub Pages.

<!-- If you double-click on the `raleigh.html` file in your folder, a map will open in your web browser. Next, let's open the `raleigh.html` file in **VS Code**. -->

This code is adapted from the `index.html` file from [this GitHub repository](https://github.com/HandsOnDataViz/leaflet-map-simple).

This exercise is an adaptation of the ["Leaflet Storymaps with Google Sheets" tutorial](https://handsondataviz.org/leaflet-storymaps-with-google-sheets.html) from the *Hands-On Data Visualization* book by Jack Dougherty & Ilya Ilyankou. The open-acess web edition of the book can be found [here](https://handsondataviz.org/).

Dougherty and Ilyankou developed Leaflet Storymaps with Google Sheets to create an open-source, easily migratable, story map tool. In this exercise, we are adapting their work with Google Sheets into using Python to create and edit CSVs to create the open-source storymap.

### Host a Live Leaflet Map on GitHub Pages

In order to host your storymap on the internet, you need to create a GitHub repository for this project. The best way to do this is to **fork** the leaflet-storymap-nr491 repo from [this repo](https://github.com/mdgaines/leaflet-storymap-nr491) on my (Mollie Gaines') GitHub.

Go to Settings > Pages and set Branch to `main` and save.

Edit your README.md to have the link to your new repo.

### Leaflet Storymap with CSVs

We are now going to start working on our Leaflet Storymap by building several `pandas dataframes` which we can save as a CSVs.

Our storymap will have a scrolling narrative that follows points along our map. You will learn how to add images, audio and video files, and different map backgrounds. 

We will build an in-class storymap to looking at different colleges and universities in Wake County.

In [121]:
import pandas as pd
import geopandas as gpd
import os
import folium
import csv

First, we will create an `Options.csv` file and save it in our `csv/` directory. We will create this CSV to look like the **Options** tab in the Google Sheet from the HODV exercise.

![GS_Options](./imgs/gs_options.PNG)

In [122]:
# Create lists for each row we are writing in our csv
header = ['Setting', 'Customize', 'Hints']
info = ['Storymap Info', '', \
    'For help, see tutorial in [HandsOnDataViz.org](https://handsondataviz.org/leaflet-storymaps-with-google-sheets.html)']
title = ['NC Counties ranked by POC, Income, and Health Outcome', '']
subtitle = ['Storymap Subtitle', \
    "Project by: Vic Abate GIS 495", "<br><small>Scroll down<i class='fafa-chevron-down'></i></small>",\
    "Add your subtitle, or delete that portion"]
logo = ['Storymap Logo', 'media/ncsu_logo.jpg', 'Path to a logo image']
google_analysis = ['Google Analytics Tracking ID', '', 'Sample format: UA-5488840-29']

map_settings = ['Map Settings', '', '']
basemap_tiles = ['Basemap Tiles', 'CartoDB.Positron',\
    '[Drop-down menu for background basemap tiles](https://leaflet-extras.github.io/leaflet-providers/preview/)']
zoom = ['Zoom Controls', 'bottomright', '']
bg_color = ['Narrative Background Color', '', '']
txt_color = ['Narrative Text Color', '', '']
lnk_color = ['Narrative Link Color', '', '']
ch_color = ['Active Chapter Background Color','','']
media_height = ['Media Container Height','300',\
    'Maximum height of the image, in pixels. 200 is default. The image will be fit into the container with its proportions kept (it won\'t be skewed).']
pixels_after_char = ['Pixels After Final Chapter','600','In pixels, at least 100']
lightbox_imgs = ['Enable Lightbox for Images','yes','yes or no']

creds = ['Credits','','']
auth_name = ['Author Name','Add your name','Appears in map credits as "View data by..." (or leave blank)']
auth_email = ['Author Email or Website','','Create link in Author Name by inserting your email or web address (or leave blank)']
auth_GH = ['Author GitHub Repo Link','https://github.com/handsondataviz/leaflet-storymaps-with-google-sheets',\
    'Insert your code repo URL to appear as link in "View code..." (or leave blank)']
code_cred = ['Code Credit','<a href="https://handsondataviz.org">HandsOnDataViz</a>',\
    'Appears in credits as "...code by..." (or leave blank)']

**NOTE:** you can have several different basemap tiles that will appear in the background. 
The options we will look at are:

![Basemaps](./imgs/gs_basemap_tiles.PNG)

We can also choose where on our map we put our zoom controls. The options are:

![zoom](./imgs/gs_zoom.PNG)

Now, we'll write our first CSV.

In [123]:
# Check that the csv directory exists
if not os.path.exists('./csv'):
    os.mkdir('./csv')

# Create a new Options.csv

# opens a csv file in write mode
# the wuth statement makes sure the file closes properly when we're done
with open('./csv/Options_Template.csv', 'w', newline='') as options:
    # create the csv writer
    writer = csv.writer(options)

    # write a row to the csv file
    writer.writerow(header)

    writer.writerow(info)
    writer.writerow(title)
    writer.writerow(subtitle)
    writer.writerow(logo)
    writer.writerow(google_analysis)

    writer.writerow(map_settings)
    writer.writerow(basemap_tiles)
    writer.writerow(zoom)
    writer.writerow(bg_color)
    writer.writerow(txt_color)
    writer.writerow(lnk_color)
    writer.writerow(ch_color)
    writer.writerow(media_height)
    writer.writerow(pixels_after_char)
    writer.writerow(lightbox_imgs)

    writer.writerow(creds)
    writer.writerow(auth_name)
    writer.writerow(auth_email)
    writer.writerow(auth_GH)
    writer.writerow(code_cred)


Now that we've saved our CSV, we will read it in as a Pandas DataFrame and make some edits so it is personalized for us.

**Note:** We could have written our original CSV with all the information we wanted, however now we can practice making edits.

In [126]:
# read in the options csv as a data frame
options_df = pd.read_csv('./csv/Options_Template.csv', index_col=0, on_bad_lines='warn')
options_df

C:\Users\Victo\AppData\Local\Temp\ipykernel_32468\1727138119.py:2: ParserWarning: Skipping line 4: expected 3 fields, saw 4

  options_df = pd.read_csv('./csv/Options_Template.csv', index_col=0, on_bad_lines='warn')


,Customize,Hints
Setting,,
Storymap Info,NaN,"For help, see tutorial in [HandsOnDataViz.org]..."
"NC Counties ranked by POC, Income, and Health Outcome",NaN,NaN
Storymap Logo,media/ncsu_logo.jpg,Path to a logo image
Google Analytics Tracking ID,NaN,Sample format: UA-5488840-29
Map Settings,NaN,NaN
Basemap Tiles,CartoDB.Positron,[Drop-down menu for background basemap tiles](...
Zoom Controls,bottomright,NaN
Narrative Background Color,NaN,NaN
Narrative Text Color,NaN,NaN


In [127]:
# we will index to the specific entry we want to edit using df[col][row] indexing.
# we read in the df with the first column as our index to make this easier for ourselves

# here, we are changing the background color of our narative bar
options_df['Customize']['Narrative Background Color'] = 'lightgray'

options_df

C:\Users\Victo\AppData\Local\Temp\ipykernel_32468\2497308199.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  options_df['Customize']['Narrative Background Color'] = 'lightgray'


,Customize,Hints
Setting,,
Storymap Info,NaN,"For help, see tutorial in [HandsOnDataViz.org]..."
"NC Counties ranked by POC, Income, and Health Outcome",NaN,NaN
Storymap Logo,media/ncsu_logo.jpg,Path to a logo image
Google Analytics Tracking ID,NaN,Sample format: UA-5488840-29
Map Settings,NaN,NaN
Basemap Tiles,CartoDB.Positron,[Drop-down menu for background basemap tiles](...
Zoom Controls,bottomright,NaN
Narrative Background Color,lightgray,NaN
Narrative Text Color,NaN,NaN


In [128]:
# Next, make edits to the Author Name and Author Github Repo Link so it has your information
options_df['Customize']['Author Name'] = 'Vic Abate'
options_df['Customize']['Credits'] = 'Dr. Tulbure, Rebecca Composto, class of GIS 495'
options_df['Customize']['Narrative Text Color'] = 'Black'
options_df.to_csv('./csv/Options.csv')

# save your options df as Options.csv


C:\Users\Victo\AppData\Local\Temp\ipykernel_32468\3528424827.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  options_df['Customize']['Author Name'] = 'Vic Abate'
C:\Users\Victo\AppData\Local\Temp\ipykernel_32468\3528424827.py:3: FutureWa

Now let's checkout what our index.html file looks like so far.

### Chapters

We will add content to our map by creating a `Chapters.csv`.

However, first we will look through our data to get the information we are interested in.

Now, let's read in some of our spatial data.

In [129]:
# Filepaths
nc_colleges_path = './data/Colleges_and_Universities/Colleges_and_Universities.shp'
nc_amtrak_stations_path = './data/Amtrak_Stations_Trains_Reproj_NC/Amtrak_Stations_Trains_Reproj_NC.shp'
county_path = './geojson/North_Carolina_State_and_County_Boundary_Polygons.geojson'

# Read data
nc_colleges = gpd.read_file(nc_colleges_path)
nc_amtrak_stations = gpd.read_file(nc_amtrak_stations_path)
county = gpd.read_file(county_path)

# Print CRS
print(nc_colleges.crs)
print(nc_amtrak_stations.crs)
print(county.crs)


EPSG:32119
EPSG:4269
EPSG:4326


In [130]:
# Reproject to WGS84
#no need to reproject geojson bc it is in the same one as 4326 alr#
nc_colleges_wgs84 = nc_colleges.to_crs(epsg=4326)
nc_amtrak_stations_wgs84 = nc_amtrak_stations.to_crs(epsg=4326)

# Create a Geo-id which is needed by the Folium (it needs to have a unique identifier for each row)
nc_colleges_wgs84['geoid'] = nc_colleges_wgs84.index.astype(str)
nc_amtrak_stations_wgs84['geoid'] = nc_amtrak_stations_wgs84.index.astype(str)

We can use the `folium` package to see what our Leaflet map will look like.

In [131]:
# Create a Map instance centered on Raleigh
m = folium.Map(location=[35.7796, -78.6382], zoom_start=8, control_scale=True)
m

In [132]:
# Show the colleges on the map

# Create college points on top of the map
for idx, row in nc_colleges_wgs84.iterrows():
    # Get lat and lon of points
    lon = row['geometry'].x
    lat = row['geometry'].y

    # Get college information
    college_name = row['NAME']

    #Add markers to the map
    folium.Marker([lat,lon], popup=college_name).add_to(m)

m

Let's parse this data down to schools in Raleigh, and remove some of the information we are not interested in.

In [133]:
nc_coll_raleigh = nc_colleges_wgs84[nc_colleges_wgs84['CITY'] == 'RALEIGH'].drop(['OBJECTID', \
    'METLNKID', 'PHONE', 'SECCLASS', 'ST_VENDOR', 'ST_VERSION', 'ADDRESS2', 'EMERGTITLE', 'EMERGPHONE',
    'EMERGEXT', 'DIRECTIONS', 'CONTDATE', 'CONTHOW', 'GEOPREC', 'QC_QA', 'onemapSDEA', \
    'GEOHOW', 'GEOLINKID', 'GEODATE', 'PHONELOC'], axis=1)
nc_coll_raleigh

,ID,FEATTYPE,NAME,ADDRESS,CITY,STATE,ZIP,ZIPP4,COUNTY,FIPS,NAICSCODE,NAICSDESCR,X,Y,DEGREE,ENROLL,geometry,geoid
0,10275649,POINT,SCHOOL OF COMMUNICATION ARTS,3000 WAKEFIELD CROSSING DRIVE,RALEIGH,NC,27614,None,WAKE,37183,611610,FINE ARTS SCHOOLS (EXCEPT ACADEMIC),-78.556242,35.942785,ASSOCIATE'S DEGREE,298,POINT (-78.55624 35.94278),0
19,10274284,POINT,WAKE TECHNICAL COMMUNITY COLLEGE,9101 FAYETTEVILLE ROAD,RALEIGH,NC,27603,5655,WAKE,37183,61151,TECHNICAL AND TRADE SCHOOLS,-78.706852,35.650636,ASSOCIATE'S DEGREE,11832,POINT (-78.70685 35.65063),19
22,10274261,POINT,PEACE COLLEGE,15 E PEACE ST,RALEIGH,NC,27604,1176,WAKE,37183,611310,COLLEGES (EXCEPT JUNIOR COLLEGES),-78.637376,35.788240,BACHELOR'S DEGREE,668,POINT (-78.63737 35.78824),22
32,10272064,POINT,SHAW UNIVERSITY,118 E SOUTH ST,RALEIGH,NC,27601,2341,WAKE,37183,611310,UNIVERSITIES,-78.637986,35.771817,MASTER'S DEGREE,2762,POINT (-78.63798 35.77182),32
49,10275987,POINT,UNIVERSITY OF PHOENIX-RALEIGH CAMPUS,5511 CAPITAL CENTER DRIVE STE. 380,RALEIGH,NC,27606,4168,WAKE,37183,611310,UNIVERSITIES,-78.729081,35.767244,MASTER'S DEGREE,221,POINT (-78.72908 35.76724),49
57,10274245,POINT,MEREDITH COLLEGE,3800 HILLSBOROUGH ST,RALEIGH,NC,27607,5237,WAKE,37183,611310,COLLEGES (EXCEPT JUNIOR COLLEGES),-78.688041,35.794525,MASTER'S DEGREE,2168,POINT (-78.68804 35.79452),57
65,10272187,POINT,SAINT AUGUSTINES COLLEGE,1315 OAKWOOD AVENUE,RALEIGH,NC,27610,2247,WAKE,37183,611310,COLLEGES (EXCEPT JUNIOR COLLEGES),-78.620682,35.784495,BACHELOR'S DEGREE,1163,POINT (-78.62068 35.78449),65
97,10274257,POINT,NORTH CAROLINA STATE UNIVERSITY AT RALEIGH,2200 HILLSBOROUGH STREET,RALEIGH,NC,27695,0001,WAKE,37183,611310,UNIVERSITIES,-78.664483,35.786472,DOCTOR'S DEGREE,30148,POINT (-78.66448 35.78647),97


We will build our storymap with each of these schools as a chapter.

Now, let's make our `Chapters.csv` with specific column headers (**do not change the headers, they work with the background Leaflet code**) and a row for each of our chapters.

In [134]:
header = ['Chapter', 'Media Link', 'Media Credit', 'Media Credit Link', 'Description', \
    'Zoom', 'Marker', 'Marker Color', 'Location', 'Latitude', 'Longitude', 'Overlay', 'Overlay Transparency', \
    'GeoJSON Overlay', 'GeoJSON Feature Properties']

# add a column of founding date to our nc_coll_raleigh df
nc_coll_raleigh['FOUNDED'] = None
nc_coll_raleigh

,ID,FEATTYPE,NAME,ADDRESS,CITY,STATE,ZIP,ZIPP4,COUNTY,FIPS,NAICSCODE,NAICSDESCR,X,Y,DEGREE,ENROLL,geometry,geoid,FOUNDED
0,10275649,POINT,SCHOOL OF COMMUNICATION ARTS,3000 WAKEFIELD CROSSING DRIVE,RALEIGH,NC,27614,None,WAKE,37183,611610,FINE ARTS SCHOOLS (EXCEPT ACADEMIC),-78.556242,35.942785,ASSOCIATE'S DEGREE,298,POINT (-78.55624 35.94278),0,None
19,10274284,POINT,WAKE TECHNICAL COMMUNITY COLLEGE,9101 FAYETTEVILLE ROAD,RALEIGH,NC,27603,5655,WAKE,37183,61151,TECHNICAL AND TRADE SCHOOLS,-78.706852,35.650636,ASSOCIATE'S DEGREE,11832,POINT (-78.70685 35.65063),19,None
22,10274261,POINT,PEACE COLLEGE,15 E PEACE ST,RALEIGH,NC,27604,1176,WAKE,37183,611310,COLLEGES (EXCEPT JUNIOR COLLEGES),-78.637376,35.788240,BACHELOR'S DEGREE,668,POINT (-78.63737 35.78824),22,None
32,10272064,POINT,SHAW UNIVERSITY,118 E SOUTH ST,RALEIGH,NC,27601,2341,WAKE,37183,611310,UNIVERSITIES,-78.637986,35.771817,MASTER'S DEGREE,2762,POINT (-78.63798 35.77182),32,None
49,10275987,POINT,UNIVERSITY OF PHOENIX-RALEIGH CAMPUS,5511 CAPITAL CENTER DRIVE STE. 380,RALEIGH,NC,27606,4168,WAKE,37183,611310,UNIVERSITIES,-78.729081,35.767244,MASTER'S DEGREE,221,POINT (-78.72908 35.76724),49,None
57,10274245,POINT,MEREDITH COLLEGE,3800 HILLSBOROUGH ST,RALEIGH,NC,27607,5237,WAKE,37183,611310,COLLEGES (EXCEPT JUNIOR COLLEGES),-78.688041,35.794525,MASTER'S DEGREE,2168,POINT (-78.68804 35.79452),57,None
65,10272187,POINT,SAINT AUGUSTINES COLLEGE,1315 OAKWOOD AVENUE,RALEIGH,NC,27610,2247,WAKE,37183,611310,COLLEGES (EXCEPT JUNIOR COLLEGES),-78.620682,35.784495,BACHELOR'S DEGREE,1163,POINT (-78.62068 35.78449),65,None
97,10274257,POINT,NORTH CAROLINA STATE UNIVERSITY AT RALEIGH,2200 HILLSBOROUGH STREET,RALEIGH,NC,27695,0001,WAKE,37183,611310,UNIVERSITIES,-78.664483,35.786472,DOCTOR'S DEGREE,30148,POINT (-78.66448 35.78647),97,None


In [135]:
nc_coll_raleigh.loc[[22], ['FOUNDED']] = 1857 # Peace College
nc_coll_raleigh.loc[[32], ['FOUNDED']] = 1865 # Shaw University
nc_coll_raleigh.loc[[65], ['FOUNDED']] = 1867 # St. Augustune's University
nc_coll_raleigh.loc[[97], ['FOUNDED']] = 1887 # NC State University
nc_coll_raleigh.loc[[57], ['FOUNDED']] = 1891 # Meredith College
nc_coll_raleigh.loc[[19], ['FOUNDED']] = 1963 # Wake Tech
nc_coll_raleigh.loc[[49], ['FOUNDED']] = 1976 # University of Phoenix
nc_coll_raleigh.loc[[0], ['FOUNDED']] = 1992 # School of communication arts

nc_coll_raleigh = nc_coll_raleigh.sort_values(by='FOUNDED')
nc_coll_raleigh

,ID,FEATTYPE,NAME,ADDRESS,CITY,STATE,ZIP,ZIPP4,COUNTY,FIPS,NAICSCODE,NAICSDESCR,X,Y,DEGREE,ENROLL,geometry,geoid,FOUNDED
22,10274261,POINT,PEACE COLLEGE,15 E PEACE ST,RALEIGH,NC,27604,1176,WAKE,37183,611310,COLLEGES (EXCEPT JUNIOR COLLEGES),-78.637376,35.788240,BACHELOR'S DEGREE,668,POINT (-78.63737 35.78824),22,1857
32,10272064,POINT,SHAW UNIVERSITY,118 E SOUTH ST,RALEIGH,NC,27601,2341,WAKE,37183,611310,UNIVERSITIES,-78.637986,35.771817,MASTER'S DEGREE,2762,POINT (-78.63798 35.77182),32,1865
65,10272187,POINT,SAINT AUGUSTINES COLLEGE,1315 OAKWOOD AVENUE,RALEIGH,NC,27610,2247,WAKE,37183,611310,COLLEGES (EXCEPT JUNIOR COLLEGES),-78.620682,35.784495,BACHELOR'S DEGREE,1163,POINT (-78.62068 35.78449),65,1867
97,10274257,POINT,NORTH CAROLINA STATE UNIVERSITY AT RALEIGH,2200 HILLSBOROUGH STREET,RALEIGH,NC,27695,0001,WAKE,37183,611310,UNIVERSITIES,-78.664483,35.786472,DOCTOR'S DEGREE,30148,POINT (-78.66448 35.78647),97,1887
57,10274245,POINT,MEREDITH COLLEGE,3800 HILLSBOROUGH ST,RALEIGH,NC,27607,5237,WAKE,37183,611310,COLLEGES (EXCEPT JUNIOR COLLEGES),-78.688041,35.794525,MASTER'S DEGREE,2168,POINT (-78.68804 35.79452),57,1891
19,10274284,POINT,WAKE TECHNICAL COMMUNITY COLLEGE,9101 FAYETTEVILLE ROAD,RALEIGH,NC,27603,5655,WAKE,37183,61151,TECHNICAL AND TRADE SCHOOLS,-78.706852,35.650636,ASSOCIATE'S DEGREE,11832,POINT (-78.70685 35.65063),19,1963
49,10275987,POINT,UNIVERSITY OF PHOENIX-RALEIGH CAMPUS,5511 CAPITAL CENTER DRIVE STE. 380,RALEIGH,NC,27606,4168,WAKE,37183,611310,UNIVERSITIES,-78.729081,35.767244,MASTER'S DEGREE,221,POINT (-78.72908 35.76724),49,1976
0,10275649,POINT,SCHOOL OF COMMUNICATION ARTS,3000 WAKEFIELD CROSSING DRIVE,RALEIGH,NC,27614,None,WAKE,37183,611610,FINE ARTS SCHOOLS (EXCEPT ACADEMIC),-78.556242,35.942785,ASSOCIATE'S DEGREE,298,POINT (-78.55624 35.94278),0,1992


In [136]:
col_names = [name.title() for name in nc_coll_raleigh.NAME]
col_names

['Peace College',
 'Shaw University',
 'Saint Augustines College',
 'North Carolina State University At Raleigh',
 'Meredith College',
 'Wake Technical Community College',
 'University Of Phoenix-Raleigh Campus',
 'School Of Communication Arts']

In [137]:
# Create a new Chapters.csv

# opens a csv file in write mode
# the wuth statement makes sure the file closes properly when we're done
with open('./csv/Chapters.csv', 'w', newline='') as options:
    # create the csv writer
    writer = csv.writer(options)

    # write a row to the csv file
    writer.writerow(header)

    writer.writerow(['Overview of Raleigh'])

    for name in col_names:
        writer.writerow([name])

In [138]:
chapter_df = pd.read_csv('./csv/Chapters.csv')
chapter_df

,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties
0,Overview of Raleigh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Peace College,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Shaw University,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Saint Augustines College,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,North Carolina State University At Raleigh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Meredith College,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Wake Technical Community College,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,University Of Phoenix-Raleigh Campus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,School Of Communication Arts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Our first chapter will give a brief overview of Raleigh. We will include a fun skyline image with a link to where we got the image (the Raleigh government webpage). Our map will show a historical, georeferenced map of the Raleigh City Plan from 1834 along with blue markers for each of the 8 schools we will look at for chapters.

To include images in our narative, we need to have them downloaded to our `media` directory.

In [139]:
chapter_df.columns

Index(['Chapter', 'Media Link', 'Media Credit', 'Media Credit Link',
       'Description', 'Zoom', 'Marker', 'Marker Color', 'Location', 'Latitude',
       'Longitude', 'Overlay', 'Overlay Transparency', 'GeoJSON Overlay',
       'GeoJSON Feature Properties'],
      dtype='object')

In [140]:
# path to the image used for this chapter
chapter_df.loc[[0], ['Media Link']] = 'media/visit_raleigh.jpg'
# Name of image source
chapter_df.loc[[0], ['Media Credit']] = 'Source: Raleigh Government'
# Link to image
chapter_df.loc[[0], ['Media Credit Link']] = 'https://raleighnc.gov/'
# Narrative description
chapter_df.loc[[0], ['Description']] = 'Before European colonization, the Raleigh area was home to indigenous \
    tribes including the Tuscarora and Occaneechi. Sir Walter Raleigh, among others, funded a group of colonists \
    to establish the <em>Cittie of Raleigh</em>. These colonists landed on Roanoke Island in <b>1587</b> and \
    later became known as the "Lost Colony" after the settlement was found abandonded in 1590. In <b>1770</b> \
    the General Assembly created Wake County. In <b>1792</b>, Raleigh was founded as the state capital of North Carolina.'
# Zoom level
chapter_df.loc[[0], ['Zoom']] = 15.75
# Markers can be Hidden, Plain, or Numbered
chapter_df.loc[[0], ['Marker']] = 'Hidden'
# if the marker is not Hidden, you can give it a color
chapter_df.loc[[0], ['Marker Color']] = ''
# name of the location
chapter_df.loc[[0], ['Location']] = 'Raleigh, NC'
chapter_df.loc[[0], ['Latitude']] = 35.7796
chapter_df.loc[[0], ['Longitude']] = -78.6382
# If you want to add an overlay of an old map you can search for a georeferenced map at
# mapwraper.net
# add the link to the map in the Overlay section 
chapter_df.loc[[0], ['Overlay']] = 'https://mapwarper.net/maps/tile/51816/{z}/{x}/{y}.png'
# Set how transparent you want the Overlay image/map to be
chapter_df.loc[[0], ['Overlay Transparency']] = 0.9


C:\Users\Victo\AppData\Local\Temp\ipykernel_32468\3844718050.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'media/visit_raleigh.jpg' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  chapter_df.loc[[0], ['Media Link']] = 'media/visit_raleigh.jpg'
C:\Users\Victo\AppData\Local\Temp\ipykernel_32468\3844718050.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Source: Raleigh Government' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  chapter_df.loc[[0], ['Media Credit']] = 'Source: Raleigh Government'
C:\Users\Victo\AppData\Local\Temp\ipykernel_32468\3844718050.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'https://raleighnc.gov/' has dtype incompatib

In [141]:
chapter_df

,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties
0,Overview of Raleigh,media/visit_raleigh.jpg,Source: Raleigh Government,https://raleighnc.gov/,"Before European colonization, the Raleigh area...",15.75,Hidden,,"Raleigh, NC",35.7796,-78.6382,https://mapwarper.net/maps/tile/51816/{z}/{x}/...,0.9,NaN,NaN
1,Peace College,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Shaw University,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Saint Augustines College,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,North Carolina State University At Raleigh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Meredith College,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Wake Technical Community College,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,University Of Phoenix-Raleigh Campus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,School Of Communication Arts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
# nc_coll_raleigh = nc_coll_raleigh.reset_index()
nc_coll_raleigh.head()

,ID,FEATTYPE,NAME,ADDRESS,CITY,STATE,ZIP,ZIPP4,COUNTY,FIPS,NAICSCODE,NAICSDESCR,X,Y,DEGREE,ENROLL,geometry,geoid,FOUNDED
22,10274261,POINT,PEACE COLLEGE,15 E PEACE ST,RALEIGH,NC,27604,1176,WAKE,37183,611310,COLLEGES (EXCEPT JUNIOR COLLEGES),-78.637376,35.788240,BACHELOR'S DEGREE,668,POINT (-78.63737 35.78824),22,1857
32,10272064,POINT,SHAW UNIVERSITY,118 E SOUTH ST,RALEIGH,NC,27601,2341,WAKE,37183,611310,UNIVERSITIES,-78.637986,35.771817,MASTER'S DEGREE,2762,POINT (-78.63798 35.77182),32,1865
65,10272187,POINT,SAINT AUGUSTINES COLLEGE,1315 OAKWOOD AVENUE,RALEIGH,NC,27610,2247,WAKE,37183,611310,COLLEGES (EXCEPT JUNIOR COLLEGES),-78.620682,35.784495,BACHELOR'S DEGREE,1163,POINT (-78.62068 35.78449),65,1867
97,10274257,POINT,NORTH CAROLINA STATE UNIVERSITY AT RALEIGH,2200 HILLSBOROUGH STREET,RALEIGH,NC,27695,0001,WAKE,37183,611310,UNIVERSITIES,-78.664483,35.786472,DOCTOR'S DEGREE,30148,POINT (-78.66448 35.78647),97,1887
57,10274245,POINT,MEREDITH COLLEGE,3800 HILLSBOROUGH ST,RALEIGH,NC,27607,5237,WAKE,37183,611310,COLLEGES (EXCEPT JUNIOR COLLEGES),-78.688041,35.794525,MASTER'S DEGREE,2168,POINT (-78.68804 35.79452),57,1891


In [143]:
for i in nc_coll_raleigh.index:
    # print(nc_coll_raleigh.iloc[i])
    school_name = nc_coll_raleigh.loc[[i], ['NAME']].values[0][0].title()
    lat = nc_coll_raleigh.loc[[i], ['Y']].values[0][0]
    lon = nc_coll_raleigh.loc[[i], ['X']].values[0][0]
    address = nc_coll_raleigh.loc[[i], ['ADDRESS']].values[0][0]

    chapter_df.loc[chapter_df['Chapter'] == school_name, 'Latitude'] = lat
    chapter_df.loc[chapter_df['Chapter'] == school_name, 'Longitude'] = lon
    chapter_df.loc[chapter_df['Chapter'] == school_name, 'Location'] = address

    chapter_df.loc[chapter_df['Chapter'] == school_name, 'Zoom'] = 16
    chapter_df.loc[chapter_df['Chapter'] == school_name, 'Marker'] = 'Plain'
    chapter_df.loc[chapter_df['Chapter'] == school_name, 'Marker Color'] = 'blue'

chapter_df


,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties
0,Overview of Raleigh,media/visit_raleigh.jpg,Source: Raleigh Government,https://raleighnc.gov/,"Before European colonization, the Raleigh area...",15.75,Hidden,,"Raleigh, NC",35.779600,-78.638200,https://mapwarper.net/maps/tile/51816/{z}/{x}/...,0.9,NaN,NaN
1,Peace College,NaN,NaN,NaN,NaN,16.00,Plain,blue,15 E PEACE ST,35.788240,-78.637376,NaN,NaN,NaN,NaN
2,Shaw University,NaN,NaN,NaN,NaN,16.00,Plain,blue,118 E SOUTH ST,35.771817,-78.637986,NaN,NaN,NaN,NaN
3,Saint Augustines College,NaN,NaN,NaN,NaN,16.00,Plain,blue,1315 OAKWOOD AVENUE,35.784495,-78.620682,NaN,NaN,NaN,NaN
4,North Carolina State University At Raleigh,NaN,NaN,NaN,NaN,16.00,Plain,blue,2200 HILLSBOROUGH STREET,35.786472,-78.664483,NaN,NaN,NaN,NaN
5,Meredith College,NaN,NaN,NaN,NaN,16.00,Plain,blue,3800 HILLSBOROUGH ST,35.794525,-78.688041,NaN,NaN,NaN,NaN
6,Wake Technical Community College,NaN,NaN,NaN,NaN,16.00,Plain,blue,9101 FAYETTEVILLE ROAD,35.650636,-78.706852,NaN,NaN,NaN,NaN
7,University Of Phoenix-Raleigh Campus,NaN,NaN,NaN,NaN,16.00,Plain,blue,5511 CAPITAL CENTER DRIVE STE. 380,35.767244,-78.729081,NaN,NaN,NaN,NaN
8,School Of Communication Arts,NaN,NaN,NaN,NaN,16.00,Plain,blue,3000 WAKEFIELD CROSSING DRIVE,35.942785,-78.556242,NaN,NaN,NaN,NaN


Now, let's save this to a CSV and see how this changes our storymap.

In [144]:
chapter_df.to_csv('./csv/Chapters.csv')

We have a map! We also have some images, links, and narrative text appearing.

Now, let's add a new image and image source information for Peace College.

In [145]:
# Add an image for our Peace College Chapter
chapter_df.loc[1, "Media Link"] = "media/peacepic.jpg"

# add source name
chapter_df.loc[1, "Media Credit"] = "Image Source: Niche"

# add the source link
chapter_df.loc[1, "Media Credit Link"] = "https://www.niche.com/colleges/william-peace-university/"

In [146]:
# Check our work
chapter_df.head()

,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties
0,Overview of Raleigh,media/visit_raleigh.jpg,Source: Raleigh Government,https://raleighnc.gov/,"Before European colonization, the Raleigh area...",15.75,Hidden,,"Raleigh, NC",35.779600,-78.638200,https://mapwarper.net/maps/tile/51816/{z}/{x}/...,0.9,NaN,NaN
1,Peace College,media/peacepic.jpg,Image Source: Niche,https://www.niche.com/colleges/william-peace-u...,NaN,16.00,Plain,blue,15 E PEACE ST,35.788240,-78.637376,NaN,NaN,NaN,NaN
2,Shaw University,NaN,NaN,NaN,NaN,16.00,Plain,blue,118 E SOUTH ST,35.771817,-78.637986,NaN,NaN,NaN,NaN
3,Saint Augustines College,NaN,NaN,NaN,NaN,16.00,Plain,blue,1315 OAKWOOD AVENUE,35.784495,-78.620682,NaN,NaN,NaN,NaN
4,North Carolina State University At Raleigh,NaN,NaN,NaN,NaN,16.00,Plain,blue,2200 HILLSBOROUGH STREET,35.786472,-78.664483,NaN,NaN,NaN,NaN


In [147]:
# Add to the Description of Peace College including when it was founded and an interesting fact about it
chapter_df.loc[1, "Description"] = "William Peace University is a private college in Raleigh, North Carolina. Affiliated with the Presbyterian Church, it offers undergraduate degrees in more than 30 majors and the School of Professional Studies offers accelerated bachelor's degrees that are online or hybrid for working adults. Wikipedia"
chapter_df.loc[1, "Media Link"] = "https://www.youtube.com/embed/aZQFQnbs_Ho"
chapter_df
chapter_df.to_csv('./csv/Chapters.csv')

Next, we'll look at adding a video to our narrative bar.

In order to add a YouTube video to our storymap, we need to first find a YouTube video we want to add. For this example, let's follow [this link](https://www.youtube.com/watch?v=aZQFQnbs_Ho) to a fan-made hype video of the NC State Football Team.

Next, click the `Share` button at the bottom right of the video and then click `Embed`. Next, **only** copy the link that appears after `src="`. We will use that in our next code chunk.

<!-- Link that should be copied: https://www.youtube.com/embed/aZQFQnbs_Ho" -->

In [148]:
# Add our copied embed youtube link to the "Media Link" column for NC State


# Save to Chapters.csv 


Let's look at our `index.html` again. What has changed? Does it look the way you expect?

### Overlay GeoJSON

Next, we'll add a GeoJSON file to our map. Leaflet uses GeoJSONs instead of Shapefiles. If you mostly use Shapefiles, just open them into QGIS and save them as GeoJSONs to get them ready for the storymap.

All of our GeoJSONs need be saved in the `geojson` folder.

**Important:** Your GeoJSON files **MUST** be in a recognized projection and have a `"crs"` attribute. *It will be easiest for you if they are all in the same CRS.*

Let's add our GeoJSON of NC State's campuses.

In [149]:
chapter_df.loc[4, "GeoJSON Overlay"] = 'geojson/North_Carolina_State_and_County_Boundary_Polygons.geojson'

# We can also set different parameters for the GeoJSON 
# These parameters are: fillColor, weight, opacity, color, and fillOpacity
chapter_df.loc[4, "GeoJSON Feature Properties"] = "fillColor:red;color:black"

# save the Chapters.csv and look at the index.html file
chapter_df.to_csv('./csv/Chapters.csv')

C:\Users\Victo\AppData\Local\Temp\ipykernel_32468\2643207555.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'geojson/North_Carolina_State_and_County_Boundary_Polygons.geojson' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  chapter_df.loc[4, "GeoJSON Overlay"] = 'geojson/North_Carolina_State_and_County_Boundary_Polygons.geojson'
C:\Users\Victo\AppData\Local\Temp\ipykernel_32468\2643207555.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'fillColor:red;color:black' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  chapter_df.loc[4, "GeoJSON Feature Properties"] = "fillColor:red;color:black"


To change a specific campus color, we will need to edit the GeoJSON itself.

*NOTE: I am working on integrating another set of Leaflet code so that you will not have to manually edit the GeoJSON files, but it's not ready yet.*

To edit the GeoJSON file, open it with a text-editor (like Notepad) or in VS Code. Use `ctrl+f` to find `"properties"`. GeoJSON files are essentially full of dictionaries. After the `"Precinct_N"` key:value pair, add `"fillColor": "blue",` and save your changes.

Refresh your `index.html`. Do you see any changes? (Note: it can take a while for changes to be fully processed. Make sure everything is saved.)

In [150]:
# Add Wake county census tracts to your map for a chapter of your choice


# Try to change the census tract for the school in your chapter to a different color

Now, let's add another historic map of Raleigh. Go to https://mapwarper.net/ and put "raleigh" in the search bar. Select "Rectified maps only" and search. 

Look through the maps and find one you want to use. Click on the "Export" tab at the top and copy the link from the "Tiles (Google/OSM scheme):" option.

<!-- https://mapwarper.net/maps/tile/44859/{z}/{x}/{y}.png -->

In [151]:
# Add this link to the Overlay column to a chapter of your choice
# remember, if the historical map does not overlap with the school for that chapter, 
# we will need to manually zoom out to see it or change our zoom settings



Try adding new images, text, vidoes, and GeoJSON files to different chapters. 
Remember to add the data we are adding to their respective folders.